In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'books'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 10766 palavras!


In [2]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :];
Ytrain, Yval = Y2[train_index], Y2[test_index];

In [3]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

#converte para outro tipo de matriz esparsa que facilita os cálculos do svm
Xtrain = Xtrain.tolil() 
Xval = Xval.tolil()

In [4]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)
# with open('app/vocabulary.txt', 'w') as out_file:
#     for word in new_vocabulary:
#         if isinstance(word, tuple):
#             s = '('+word[0]+","+word[1]+')'+'\n'
#             out_file.write(s)
#         else:
#             out_file.write(word+'\n')

Número de features antes do chi-quadrado: 10766
----------------------------------------
Número de features após chi-quadrado: 258
['bare', ('not', 'come'), 'bias', ('not', 'enough'), 'darl', 'write', ('not', 'work'), 'familiar', ('not', 'sure'), ('not', 'alway'), 'amaz', 'danni', 'provid', 'better', 'swisher', ('not', 'know'), 'advanc', 'superfici', 'buy', ('not', 'recommend'), ('no', 'longer'), ('not', 'expect'), 'trotski', 'not', 'becki', ('not', 'believ'), 'believ', 'bore', 'unfortun', 'even', 'british', 'info', ('not', 'get'), 'twentieth', 'whine', 'stare', 'slow', 'super', 'prospect', 'actual', 'expand', 'highli', 'lie', ('not', 'find'), ('not', 'read'), ('not', 'complet'), 'seem', 'secondari', 'entertain', 'unseen', 'enjoy', 'sean', 'poorli', 'save', ('not', 'go'), 'spend', 'comprehens', 'famou', 'simpl', ('not', 'want'), 'aw', 'inform', ('not', 'thought'), 'gpu', 'shaara', 'left', 'great', 'think', 'evil', 'love', 'half', 'suppos', 'determin', 'disjoint', 'ridicul', ('not', 'bo

In [5]:
import svmutil
from svmutil import svm_read_problem
from svmutil import svm_problem
from svmutil import svm_parameter
from svmutil import svm_train
from svmutil import svm_predict
from svmutil import svm_save_model

def gridSearch(X, Y, Xval, Yval):
    #inicializa as variáveis que deverão ser retornadas pela função
    custo = 1000
    gamma = 1000
    
    values = []
    for i in range(1, 10):
         for j in range(-1, 3):
            values.append(i/(10 ** j))
    
    acr = 0
    
    for i in values:
        for j in values:
            print(i)
            print(j)
            model = svm_train(Y, X, '-c %f -g %f -q' %(i, j))
            classes = svm_predict([], Xval, model, '-q')
            acuracia = np.sum(np.array(classes[0])==Yval)/len(Yval)
            if acr < acuracia:
                custo = i
                gamma = j
                acr = acuracia
                print(acr)
    
    return custo, gamma

In [6]:
def svm(Xtrain,Ytrain,Xval,Yval) :

    #bestC, bestGamma = gridSearch(Xtrain, Ytrain, Xval, Yval)

    bestC = 10
    bestGamma = 0.01
    
    kernel = 2 #kernel radial
    model = svm_train(Ytrain, Xtrain, '-c %f -t %d -g %f -q' %(bestC, kernel, bestGamma))
    
    #svm_save_model('app/test.model', model)
    
    classes = svm_predict([], Xval, model, '-q')
    acuracia = np.sum(np.array(classes[0])==Yval)/len(Yval)

    return np.array(classes[0], dtype="int"), acuracia

In [7]:
Ypred, acuracia = svm(Xtrain,Ytrain,Xval,Yval)
print("Acurácia é "+ str(acuracia*100))

Acurácia é 92.75


In [8]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)


	Revocacao   Precisao   F-medida   Classe
	0.946       0.915      0.930      0
	0.908       0.941      0.924      1
	------------------------------------------------
	0.927       0.928      0.928      Média macro
	0.927       0.927      0.927      Média micro

	Acuracia: 0.927


In [9]:
def curva_aprendizado(X, Y, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y), 1):
           
        Ypred, acuracia = svm(X[:i],Y[:i],X[:i],Y[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = svm(X[:i],Y[:i],Xval,Yval)
        perf_val.append(acuracia)

    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()



In [10]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)